# Apriori algorithm

Implémentation de l'algorithme Apriori sur notre jeu de données. Les métriques calculées sont expliquées ci-dessous, avec des explications adaptées à notre cas d'utilisation et inspirée du tutoriel suivant : https://towardsdatascience.com/apriori-algorithm-for-association-rule-learning-how-to-find-clear-links-between-transactions-bf7ebc22cf0a

## Support
La première étape consiste à trouver les GHM fréquents. Le calcul du support est basé sur la fréquence : \
\
$Support(GHM_i) = \frac{\text{Nb d'hospitalisations pour le } GHM_i}{\text{Nb total d'hospitalisations}}$

## Confidence
La confiance nous indique à quel point nous pouvons être sûrs (d'après nos données) qu'un GHM se trouvera dans le parcours de soins d'un patient, étant donné qu'un autre GHM a été observé.\
\
$Confidence(GHM_i\longrightarrow GHM_j) = \frac{Support(GHM_i \cup GHM_j)}{Support(GHM_i)}$

## Lift

Lift est une mesure qui nous indique si la probabilité d'apparition d'un GHM augmente ou diminue compte tenu de la présence d'un autre GHM dans le parcours de soin:\
\
$Lift(GHM_i \longrightarrow GHM_j) = \frac{\mathbb{P}(GHM_i \cup GHM_j)}{Support(GHM_i) \times Support(GHM_j)}$ \
\
On peut simplifier cette formule en remplaçant
$\frac{\mathbb{P}(GHM_i \cup GHM_j)}{Support(GHM_i)}$
par $Confidence(GHM_i\longrightarrow GHM_j)$. Ansi, on obtient : \
\
$Lift(GHM_i \longrightarrow GHM_j) = \frac{Confidence(GHM_i\longrightarrow GHM_j)}{Support(GHM_j)}$

In [1]:
!pip install apyori

  Preparing metadata (setup.py) ... done
  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5955 sha256=7283bdfc1bc45406f6650b5a815aa42209e7db124d697bf984a5eef61204be9e
  Stored in directory: /home/onyxia/.cache/pip/wheels/c4/1a/79/20f55c470a50bb3702a8cb7c94d8ada15573538c7f4baebe2d
Successfully built apyori


In [2]:
from apyori import apriori
import pandas as pd
import pickle

In [3]:
def generate_output(db):
    association_rules = apriori(db,
                            min_support=0.02,
                            min_confidence=0.05,
                            #min_lift=1,
                            max_length=None)
    
    association_results = list(association_rules)
    
    listRules = [list(association_results[i][0]) for i in range(0,len(association_results))]
    listSupport = [association_results[i].support for i in range(0,len(association_results))]
    
    return pd.DataFrame({"Motif":listRules, "Support":listSupport}).sort_values(by="Support", ascending=False).reset_index(drop=True)

In [4]:
%%capture capt
p_soins=pd.read_csv("../data/parcours_soins.csv")
patients=pd.read_csv("../data/profil_patient.csv")

In [5]:
p_soins=p_soins.rename(columns={"BEN_NIR_IDT": "CODE_PATIENT"})
p_soins=p_soins.set_index("CODE_PATIENT")
patients=patients.set_index("CODE_PATIENT")
p_soins["cluster"]=patients["cluster"]

/tmp/ipykernel_294/3737135568.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  p_soins["cluster"]=patients["cluster"]


In [6]:
with open('../data/parcours_soins.pickle', 'rb') as handle:
    p_soin=pickle.load(handle)
    
with open('../data/parcours_soins_dp.pickle', 'rb') as handle:
    p_soin_dp=pickle.load(handle)

In [7]:
corpus = []
for i in range(18):
    corpus += p_soin[f"Cluster {i}"]

In [8]:
len(corpus)

10051

In [9]:
generate_output(corpus)

,Motif,Support
0,[05M092],0.182867
1,[05M093],0.162272
2,[02C05J],0.138295
3,[05K101],0.098000
4,[04M053],0.085066
5,[05M091],0.079893
6,[05M09T],0.078599
7,[23M20Z],0.066063
8,[23M20T],0.058104
9,[06K04J],0.055218
